# Assignment 8

In [260]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

### Set up connection to WRDS...

In [210]:
#sign in

import wrds
db=wrds.Connection(wrds_username='sevogel')
#db.create_pgpass_file()


Loading library list...
Done


### Download and format data

In [211]:
import numpy as np
import datetime
import pandas as pd
import datetime
#################################
# Get data
#################################


#use own index instead
# get index returns

# get stock returns
crsp_m = db.raw_sql("""
                      select a.permno, a.date, 
                      b.shrcd, b.exchcd,
                      a.ret, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1970' and '12/31/2019'
                      and b.exchcd between 1 and 2
                      and b.shrcd between 10 and 11
                      """, date_cols=['date']) 
msf=crsp_m.drop(['shrcd','exchcd'],axis=1)
del(crsp_m) 




#variables
msf['permno'] = msf['permno'].astype(int)
msf['size'] = msf['shrout'] * msf['prc'].abs()
msf['const'] = 1
msf=msf.sort_values(['permno','date'])
msf['size_lag']= msf.groupby('permno')['size'].shift(1).to_frame()




In [212]:
# save data
msf.to_csv('data4.csv', sep=';', index=False)


In case memory crashes, take a break here and load only the necessary data below...

In [2]:
import numpy as np
import pandas as pd

# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan
    
import numpy as np
import pandas as pd

# load data
data = pd.read_csv("data4.csv", sep=';')
data['date']=pd.to_datetime(data['date'], format='%Y-%m-%d')
#data=data.drop(['rf','mprem','exret','obs'],axis=1)



#load fama-french factors and momentum
factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.txt',delimiter="\s+")
factors=factors.reset_index()
factors=factors.rename(columns={"index": "date"})

mom = pd.read_csv('F-F_Momentum_Factor.txt',delimiter="\s+")
mom=mom.reset_index()
mom=mom.rename(columns={"index": "date"})

# merge with crsp data
factors["year"]=factors["date"].astype(str).str.slice(stop=4).astype(int)
factors["month"]=factors["date"].astype(str).str.slice(start=-2).astype(int)

mom["year"]=mom["date"].astype(str).str.slice(stop=4).astype(int)
mom["month"]=mom["date"].astype(str).str.slice(start=-2).astype(int)


data["date"]=data["date"].dt.strftime("%Y-%m-%d")
data["year"]=data["date"].str.slice(stop=4).astype(int)
data["month"]=data["date"].str.slice(start=5, stop=7).astype(int)

del factors['date']
del mom['date']

factors["Mkt-RF"]=factors["Mkt-RF"]/100
factors["SMB"]=factors["SMB"]/100
factors["HML"]=factors["HML"]/100
factors["RMW"]=factors["RMW"]/100
factors["CMA"]=factors["CMA"]/100
factors["RF"]=factors["RF"]/100
mom["Mom"]=mom["Mom"]/100

data=pd.merge(data,factors, on=["year","month"],how="left")
data=pd.merge(data,mom, on=["year","month"],how="left")

factors=factors.rename(columns={"index": "date"})


# If price data is used, one needs to use the absolute values (some observations are negative)

data['prc']=np.abs(data['prc'])
data = data[data['ret'].notnull()]
obs=data[['const','permno']].groupby(['permno']).sum().reset_index().rename(columns={'const': 'obs'})
data = pd.merge(data, obs, how='left', left_on=['permno'],right_on=['permno'])

data=data.rename(columns={ "RF": "rf","Mkt-RF": "mprem"})
data['exret']=data['ret']-data['rf']
data['ret_lag']= data.groupby('permno')['ret'].shift(2).to_frame()
#data=data[data['obs']==240]
################# rolling windows ####################

# in order to keep track of the dates, it is convenient to define an index
data = data.sort_values(['permno', 'date'])
data=data.set_index('date')

roll_rets=data.groupby('permno')['ret_lag'].rolling(11).mean().reset_index()
roll_rets=roll_rets.rename(columns={"ret_lag": "ret_lag_roll"})

#merge new values with original dataset
data=data.reset_index()

data = pd.merge(data, roll_rets, how='left', left_on=['permno','date'],right_on=['permno','date'])
data=data[(data['ret_lag_roll'].notnull())]
# drop all observations with missing values
data=data[(data['ret_lag_roll'].notnull())]

### Calculate deciles, get portfolio returns

In [3]:




#################################
# Compute deciles
#################################
               
data['decile']=1+data[['ret_lag_roll','date']].groupby('date')\
.transform(lambda x: pd.qcut(x, 10, labels=False))


data = data.sort_values(['decile', 'date'])

#################################
# Compute value-weighted Returns and Betas #
#################################



# value weigthed return
vwrets=data.groupby(['decile','date']).apply(wavg, 'ret','size_lag')\
.to_frame().reset_index().rename(columns={0: 'vwret'})

##########

vwdata=pd.merge(vwrets, data[['date','rf','mprem','HML','SMB','RMW','CMA','Mom']].drop_duplicates(), how='left', left_on=['date'],right_on='date')
vwdata['vwexret']=vwdata['vwret']-vwdata['rf']
covariances=vwdata.groupby('decile')['vwexret'].cov(vwdata['mprem']).to_frame().reset_index().rename(columns={'vwexret': 'covar'})
mvar=vwdata.groupby('decile')['mprem'].var().to_frame().reset_index().rename(columns={'mprem': 'varm'})

#merge stuff
vwbetas = pd.merge(covariances, mvar, how='left', left_on=['decile'],right_on='decile')
# cumpute the betas
vwbetas['beta']=vwbetas['covar']/vwbetas['varm']
#drop some stuff that is not needed anymore
vwbetas=vwbetas.drop(['covar', 'varm'],axis=1)



avgvwrets=vwrets.groupby(['decile']).mean().reset_index()

# convert data back to what we had before
data=data.reset_index()

#################################
# Compute Portfolio Returns #
#################################


HML_ret_vw=vwrets.loc[vwrets.decile ==10]['vwret'].values \
-vwrets.loc[vwrets.decile ==1]['vwret'].values\




# make regressions

import statsmodels.regression.linear_model as lm

lab_columns = ['alpha','beta','HML','SMB']
lab_ind_tmp = np.array(range(0,4))


df_data = pd.DataFrame(data=np.empty((4, 4)), columns=lab_columns,index=lab_ind_tmp)

y = HML_ret_vw
x = vwdata[vwdata['decile']==1][['mprem','HML','SMB']].values
X = lm.add_constant(x)
x2 = vwdata[vwdata['decile']==1][['mprem',]].values
X2 = lm.add_constant(x2)



model = lm.OLS(endog=y,exog=X,missing='drop')
results = model.fit()
model2 = lm.OLS(endog=y,exog=X2,missing='drop')
results2 = model2.fit()
    
# update table
#df_data.loc[i].iloc[0:4] = np.concatenate((results.params,results.params))
df_data.loc[0].iloc[0:4] = np.around(results.params,3)
df_data.loc[1].iloc[0:4] = \
        np.around(np.divide(results.params,results.HC0_se),2)
df_data.loc[2].iloc[0:2] = np.around(results2.params,3)
df_data.loc[3].iloc[0:2] = \
        np.around(np.divide(results2.params,results2.HC0_se),2)
display(df_data)


,alpha,beta,HML,SMB
0,0.016,-0.405,-6.200000e-01,1.100000e-02
1,5.390,-4.000,-3.800000e+00,7.000000e-02
2,0.014,-0.299,1.444230e-310,1.444230e-310
3,4.590,-3.060,1.444230e-310,1.444230e-310


third row: alpha and beta wrt market
fourth row: t-stats for that

first row: coefficients wrt 3 factor model
second row: t-stats on that


# double sorting

In [4]:
import numpy as np
import pandas as pd

# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan
    
import numpy as np
import pandas as pd

# load data
data = pd.read_csv("data4.csv", sep=';')
data['date']=pd.to_datetime(data['date'], format='%Y-%m-%d')
#data=data.drop(['rf','mprem','exret','obs'],axis=1)



#load fama-french factors and momentum
factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.txt',delimiter="\s+")
factors=factors.reset_index()
factors=factors.rename(columns={"index": "date"})

mom = pd.read_csv('F-F_Momentum_Factor.txt',delimiter="\s+")
mom=mom.reset_index()
mom=mom.rename(columns={"index": "date"})

# merge with crsp data
factors["year"]=factors["date"].astype(str).str.slice(stop=4).astype(int)
factors["month"]=factors["date"].astype(str).str.slice(start=-2).astype(int)

mom["year"]=mom["date"].astype(str).str.slice(stop=4).astype(int)
mom["month"]=mom["date"].astype(str).str.slice(start=-2).astype(int)


data["date"]=data["date"].dt.strftime("%Y-%m-%d")
data["year"]=data["date"].str.slice(stop=4).astype(int)
data["month"]=data["date"].str.slice(start=5, stop=7).astype(int)

del factors['date']
del mom['date']

factors["Mkt-RF"]=factors["Mkt-RF"]/100
factors["SMB"]=factors["SMB"]/100
factors["HML"]=factors["HML"]/100
factors["RMW"]=factors["RMW"]/100
factors["CMA"]=factors["CMA"]/100
factors["RF"]=factors["RF"]/100
mom["Mom"]=mom["Mom"]/100

data=pd.merge(data,factors, on=["year","month"],how="left")
data=pd.merge(data,mom, on=["year","month"],how="left")

factors=factors.rename(columns={"index": "date"})


# If price data is used, one needs to use the absolute values (some observations are negative)

data['prc']=np.abs(data['prc'])
data = data[data['ret'].notnull()]
obs=data[['const','permno']].groupby(['permno']).sum().reset_index().rename(columns={'const': 'obs'})
data = pd.merge(data, obs, how='left', left_on=['permno'],right_on=['permno'])

data=data.rename(columns={ "RF": "rf","Mkt-RF": "mprem"})
data['exret']=data['ret']-data['rf']
data['ret_lag']= data.groupby('permno')['ret'].shift(2).to_frame()
#data=data[data['obs']==240]
################# rolling windows ####################

# in order to keep track of the dates, it is convenient to define an index
data = data.sort_values(['permno', 'date'])
data=data.set_index('date')

roll_rets=data.groupby('permno')['ret_lag'].rolling(11).mean().reset_index()
roll_rets=roll_rets.rename(columns={"ret_lag": "ret_lag_roll"})

#merge new values with original dataset
data=data.reset_index()

data = pd.merge(data, roll_rets, how='left', left_on=['permno','date'],right_on=['permno','date'])
data=data[(data['ret_lag_roll'].notnull())]
# drop all observations with missing values
data=data[(data['ret_lag_roll'].notnull())]

In [5]:

#################################
# Compute deciles
#################################
data=data[(data['ret'].notnull())&(data['size_lag'].notnull())].copy()

data['size_group']=1+data[['size_lag','date']].groupby('date')\
.transform(lambda x: pd.qcut(x, 2, labels=False))


data['decile']=1+data[['ret_lag_roll','date','size_group']].groupby(['date','size_group'])\
.transform(lambda x: pd.qcut(x, 10, labels=False))


data = data.sort_values(['size_group','decile', 'date'])

#################################
# Compute value-weighted Returns and Betas #
#################################



# value weigthed return
vwrets=data.groupby(['size_group','decile','date']).apply(wavg, 'ret','size_lag')\
.to_frame().reset_index().rename(columns={0: 'vwret'})

##########

vwdata=pd.merge(vwrets, data[['date','rf','mprem','HML','SMB','RMW','CMA','Mom']].drop_duplicates(), how='left', left_on=['date'],right_on='date')
vwdata['vwexret']=vwdata['vwret']-vwdata['rf']
covariances=vwdata.groupby(['size_group','decile'])['vwexret'].cov(vwdata['mprem']).to_frame().reset_index().rename(columns={'vwexret': 'covar'})
mvar=vwdata.groupby(['size_group','decile'])['mprem'].var().to_frame().reset_index().rename(columns={'mprem': 'varm'})

#merge stuff
vwbetas = pd.merge(covariances, mvar, how='left', left_on=['decile'],right_on='decile')
# cumpute the betas
vwbetas['beta']=vwbetas['covar']/vwbetas['varm']
#drop some stuff that is not needed anymore
vwbetas=vwbetas.drop(['covar', 'varm'],axis=1)



avgvwrets=vwrets.groupby(['size_group','decile']).mean().reset_index()

# convert data back to what we had before
data=data.reset_index()

#################################
# Compute Portfolio Returns #
#################################


HML_ret_vw_small1=vwrets[(vwrets['decile'] ==10) &(vwrets['size_group'] ==1)]['vwret'].values
HML_ret_vw_large1=vwrets[(vwrets['decile'] ==10) &(vwrets['size_group'] ==2)]['vwret'].values
HML_ret_vw_small2=vwrets[(vwrets['decile'] ==1) &(vwrets['size_group'] ==1)]['vwret'].values
HML_ret_vw_large2=vwrets[(vwrets['decile'] ==1) &(vwrets['size_group'] ==2)]['vwret'].values
HML_ret_vw_small=HML_ret_vw_small1-HML_ret_vw_small2
HML_ret_vw_large=HML_ret_vw_large1-HML_ret_vw_large2


# make regressions



# Table for small stocks

In [6]:
import statsmodels.regression.linear_model as lm

lab_columns = ['alpha','beta','HML','SMB','MOM']
lab_ind_tmp = np.array(range(0,4))


df_data = pd.DataFrame(data=np.empty((4, 5)), columns=lab_columns,index=lab_ind_tmp)

y = HML_ret_vw_small
x = vwdata[(vwdata['decile']==1)&(vwdata['size_group']==1)][['mprem','HML','SMB','Mom']].values
X = lm.add_constant(x)
x2 = vwdata[(vwdata['decile']==1)&(vwdata['size_group']==1)][['mprem',]].values
X2 = lm.add_constant(x2)



model = lm.OLS(endog=y,exog=X,missing='drop')
results = model.fit()
model2 = lm.OLS(endog=y,exog=X2,missing='drop')
results2 = model2.fit()
    
# update table
#df_data.loc[i].iloc[0:4] = np.concatenate((results.params,results.params))
df_data.loc[0].iloc[0:5] = np.around(results.params,3)
df_data.loc[1].iloc[0:5] = \
        np.around(np.divide(results.params,results.HC0_se),2)
df_data.loc[2].iloc[0:2] = np.around(results2.params,3)
df_data.loc[3].iloc[0:2] = \
        np.around(np.divide(results2.params,results2.HC0_se),2)
display(df_data)



,alpha,beta,HML,SMB,MOM
0,0.008,0.010,-0.047000,-0.221000,1.398000
1,3.560,0.170,-0.490000,-2.410000,18.340000
2,0.018,-0.232,0.009618,0.008384,0.009238
3,5.790,-2.400,0.010862,0.011023,0.013248


third row: alpha and beta wrt market
fourth row: t-stats for that

first row: coefficients wrt 3 factor model
second row: t-stats on that

# Table for large stocks

In [267]:

df_data = pd.DataFrame(data=np.empty((4, 5)), columns=lab_columns,index=lab_ind_tmp)

y = HML_ret_vw_large
x = vwdata[(vwdata['decile']==1)&(vwdata['size_group']==1)][['mprem','HML','SMB','Mom']].values
X = lm.add_constant(x)
x2 = vwdata[(vwdata['decile']==1)&(vwdata['size_group']==1)][['mprem',]].values
X2 = lm.add_constant(x2)



model = lm.OLS(endog=y,exog=X,missing='drop')
results = model.fit()
model2 = lm.OLS(endog=y,exog=X2,missing='drop')
results2 = model2.fit()
    
# update table
#df_data.loc[i].iloc[0:4] = np.concatenate((results.params,results.params))
df_data.loc[0].iloc[0:5] = np.around(results.params,3)
df_data.loc[1].iloc[0:5] = \
        np.around(np.divide(results.params,results.HC0_se),2)
df_data.loc[2].iloc[0:2] = np.around(results2.params,3)
df_data.loc[3].iloc[0:2] = \
        np.around(np.divide(results2.params,results2.HC0_se),2)
display(df_data)


,alpha,beta,HML,SMB,MOM
0,-0.001,0.061,-7.800000e-02,2.150000e-01,1.308000e+00
1,-1.120,1.830,-1.320000e+00,4.390000e+00,2.903000e+01
2,0.008,-0.089,1.018558e-312,1.018558e-312,2.371515e-322
3,3.000,-1.040,1.018558e-312,1.018558e-312,2.371515e-322


third row: alpha and beta wrt market
fourth row: t-stats for that

first row: coefficients wrt 3 factor model
second row: t-stats on that
